# Problem 1

In [1]:
def hash_join_problem1(R1, R2):
    # Build a hash table on R2 using attribute B as the key.
    # For each value of B, we store all full tuples from R2 that have that B.
    h = {}
    for t in R2:                
        b = t[0]
        if b not in h:
            h[b] = []
        h[b].append(t)         

    # Now we scan R1 and look up matches in the hash table.
    result = []
    for (a, b) in R1:
        if b in h:
            # For each matching R2 tuple, produce the joined (A, B, C)
            for (_, c) in h[b]:
                result.append((a, b, c))
    return result


# Example dataset: 10 tuples in R1 and 10 in R2 
R1 = [(1,12),(2,5),(3,10),(4,17),(5,8),(6,19),(7,16),(8,8),(9,20),(10,19)]
R2 = [(8,100),(7,101),(19,102),(3,103),(1,104),(19,105),(11,106),(11,107),(20,108),(20,109)]

# Run the join
output = hash_join_problem1(R1, R2)
output

[(5, 8, 100),
 (6, 19, 102),
 (6, 19, 105),
 (8, 8, 100),
 (9, 20, 108),
 (9, 20, 109),
 (10, 19, 102),
 (10, 19, 105)]

# Problem 2

In [2]:
# q(A1, A2, ..., Ak+1) :- R1(A1,A2), R2(A2,A3), ..., Rk(Ak, Ak+1)

from collections import defaultdict

def semijoin_reduce(left, right, join_attr_left, join_attr_right):
   
    right_keys = {t[join_attr_right] for t in right}
    return [t for t in left if t[join_attr_left] in right_keys]


def yannakakis_line_join(relations):
    """
    Input:
        relations = [R1, R2, ..., Rk]
        where Ri is a list of tuples (Ai, Ai+1)

    Output:
        The full join result of the k-line path.
    """

    k = len(relations)

   
    # 1. Bottom-up pass (semijoins)
    
    # Start from Rk and move backwards to R1
    reduced = relations.copy()

    for i in range(k - 1, 0, -1):
        # Join attribute:
        # Ri joins with Ri+1 on Ai+1
        reduced[i-1] = semijoin_reduce(
            left=reduced[i-1],
            right=reduced[i],
            join_attr_left=1,   # Ai+1 in Ri
            join_attr_right=0   # Ai+1 in Ri+1
        )

   
    # 2. Top-down pass (semijoins)
   
    for i in range(k - 1):
        reduced[i+1] = semijoin_reduce(
            left=reduced[i+1],
            right=reduced[i],
            join_attr_left=0,   # Ai+1 in Ri+1
            join_attr_right=1   # Ai+1 in Ri
        )

    
    # 3. Final join (now all small)
    
    # Start building the result from R1
    result = [(a1, a2) for (a1, a2) in reduced[0]]  # (A1,A2)

    # Extend step-by-step through remaining relations
    for i in range(1, k):
        next_rel = reduced[i]
        
        # Build an index for fast lookups
        index = defaultdict(list)
        for (x, y) in next_rel:
            index[x].append(y)

        new_result = []
        for tup in result:
            last_val = tup[-1]  # the Ai+1 value
            if last_val in index:
                for nxt in index[last_val]:
                    new_result.append(tup + (nxt,))
        result = new_result

    return result



# Example: 3-line join (R1, R2, R3)

R1 = [(1,2), (2,3), (3,4), (4,5)]
R2 = [(2,10), (3,11), (4,12), (5,13)]
R3 = [(10,100), (11,101), (12,102), (13,103)]

relations = [R1, R2, R3]

output = yannakakis_line_join(relations)
output


[(1, 2, 10, 100), (2, 3, 11, 101), (3, 4, 12, 102), (4, 5, 13, 103)]

# Problem 3

In [3]:
# q(A1, ..., Ak+1) :- R1(A1,A2), R2(A2,A3), ..., Rk(Ak,Ak+1)

from collections import defaultdict

# We reuse the hash join idea from Problem 1.
def hash_join_two(rel_left, rel_right):
    """
    Join two binary relations:
    rel_left:  list of tuples (..., x)
    rel_right: list of tuples (x, y)
    Output: joined tuples (..., x, y)
    """
    # Build a hash table on the first attribute of rel_right.
    h = defaultdict(list)
    for (x, y) in rel_right:
        h[x].append(y)

    result = []
    for t in rel_left:
        x = t[-1]  # the last attribute is the join key
        if x in h:
            for y in h[x]:
                result.append(t + (y,))
    return result


def naive_line_join(relations):
    """
    Input:
        relations = [R1, R2, ..., Rk]
        where Ri is a list of tuples (Ai, Ai+1)

    Output:
        All joined tuples from R1 ⋈ R2 ⋈ ... ⋈ Rk
    """

    # Start by converting R1(A1,A2) into tuples (A1,A2)
    result = [(a1, a2) for (a1, a2) in relations[0]]

    # Iteratively join with R2, R3, ..., Rk
    for i in range(1, len(relations)):
        result = hash_join_two(result, relations[i])

    return result



# Example dataset for a 3-line join

R1 = [(1,2), (2,3), (3,4), (4,5)]
R2 = [(2,10), (3,11), (4,12), (5,13)]
R3 = [(10,100), (11,101), (12,102), (13,103)]

relations = [R1, R2, R3]

output_problem3 = naive_line_join(relations)
output_problem3


[(1, 2, 10, 100), (2, 3, 11, 101), (3, 4, 12, 102), (4, 5, 13, 103)]

# Problem 4

In [4]:
import random
import time

# Create the 3 relations 
# R1: (i, x) for i=1..100, x random in [1,5000]
R1_100 = [(i, random.randint(1, 5000)) for i in range(1, 101)]

# R2: (y, j) for j=1..100, y random in [1,5000]
R2_100 = [(random.randint(1, 5000), j) for j in range(1, 101)]

# R3: (ℓ, ℓ) for ℓ=1..100
R3_100 = [(l, l) for l in range(1, 101)]

relations_3line = [R1_100, R2_100, R3_100]


#Run Problem 2
start_p2 = time.perf_counter()
output_p2 = yannakakis_line_join(relations_3line)
time_p2 = time.perf_counter() - start_p2

#Run Problem 3
start_p3 = time.perf_counter()
output_p3 = naive_line_join(relations_3line)
time_p3 = time.perf_counter() - start_p3


same_output = set(output_p2) == set(output_p3)
len_output = len(output_p2)

# Return timing + correctness check + output size
time_p2, time_p3, same_output, len_output


(0.0003056996501982212, 0.00037400005385279655, True, 2)

# Problem 5

In [5]:

import random, time

# Construct R1
R1_p5 = []
R1_p5 += [(i,5) for i in range(1,1001)]
R1_p5 += [(i,7) for i in range(1001,2001)]
R1_p5.append((2001,2002))
random.shuffle(R1_p5)

# Construct R2
R2_p5 = []
R2_p5 += [(5,i) for i in range(1,1001)]
R2_p5 += [(7,i) for i in range(1001,2001)]
R2_p5.append((2002,8))
random.shuffle(R2_p5)

# Construct R3
R3_p5 = [(random.randint(2002,3000), random.randint(1,3000)) for _ in range(2000)]
R3_p5.append((8,30))
random.shuffle(R3_p5)

relations_p5 = [R1_p5, R2_p5, R3_p5]

# Run Problem 2
t0 = time.perf_counter()
out_p2 = yannakakis_line_join(relations_p5)
t1 = time.perf_counter()

# Run Problem 3
t2 = time.perf_counter()
out_p3 = naive_line_join(relations_p5)
t3 = time.perf_counter()

(len(out_p2), len(out_p3), t1-t0, t3-t2)


(1001, 1001, 0.004727200139313936, 0.8710366999730468)

# Problem 7

In [10]:
import pandas as pd

R1 = pd.read_csv(r"QueryRelations\R1.csv", skiprows=1, header=None, names=["A1","A2"])
R2 = pd.read_csv(r"QueryRelations\R2.csv", skiprows=1, header=None, names=["A2","A3"])
R3 = pd.read_csv(r"QueryRelations\R3.csv", skiprows=1, header=None, names=["A1","A3"])
R4 = pd.read_csv(r"QueryRelations\R4.csv", skiprows=1, header=None, names=["A3","A4"])
R5 = pd.read_csv(r"QueryRelations\R5.csv", skiprows=1, header=None, names=["A4","A5"])
R6 = pd.read_csv(r"QueryRelations\R6.csv", skiprows=1, header=None, names=["A5","A6"])
R7 = pd.read_csv(r"QueryRelations\R7.csv", skiprows=1, header=None, names=["A4","A6"])

#R1.head(), R2.head(), R3.head(), R4.head(), R5.head(), R6.head(), R7.head()


In [11]:
def generic_join_query(R1,R2,R3,R4,R5,R6,R7):
    R1t = [tuple(x) for x in R1.to_numpy()]
    R2t = [tuple(x) for x in R2.to_numpy()]
    R3t = [tuple(x) for x in R3.to_numpy()]
    R4t = [tuple(x) for x in R4.to_numpy()]
    R5t = [tuple(x) for x in R5.to_numpy()]
    R6t = [tuple(x) for x in R6.to_numpy()]
    R7t = [tuple(x) for x in R7.to_numpy()]

    out = []

    for (a1,a2) in R1t:
        for (a2b,a3) in R2t:
            if a2 != a2b: continue
            for (a1b,a3b) in R3t:
                if a1 != a1b or a3 != a3b: continue
                for (a3c,a4) in R4t:
                    if a3 != a3c: continue
                    for (a4b,a5) in R5t:
                        if a4 != a4b: continue
                        for (a5b,a6) in R6t:
                            if a5 != a5b: continue
                            for (a4c,a6b) in R7t:
                                if a4 == a4c and a6 == a6b:
                                    out.append((a1,a2,a3,a4,a5,a6))
    return out


In [12]:
def ghw_join(R1,R2,R3,R4,R5,R6,R7):

    J1 = R1.merge(R2, on="A2", how="inner")
    J2 = J1.merge(R3, on=["A1","A3"], how="inner")
    J3 = J2.merge(R4, on="A3", how="inner")
    J4 = J3.merge(R5, on="A4", how="inner")
    J5 = J4.merge(R6, on="A5", how="inner")
    J6 = J5.merge(R7, on=["A4","A6"], how="inner")

    return J6


In [13]:
def fhw_join(R1,R2,R3,R4,R5,R6,R7):
    return ghw_join(R1,R2,R3,R4,R5,R6,R7)

In [14]:
# Fix for dtype mismatch across A1..A6 to avoid merge errors
cols = ["A1","A2","A3","A4","A5","A6"]

def normalize_df(df):
    for c in df.columns:
        df[c] = df[c].astype(str).astype(int)
    return df

R1 = normalize_df(R1)
R2 = normalize_df(R2)
R3 = normalize_df(R3)
R4 = normalize_df(R4)
R5 = normalize_df(R5)
R6 = normalize_df(R6)
R7 = normalize_df(R7)

R1.dtypes


A1    int32
A2    int32
dtype: object

In [15]:
import time

# Generic Join
t0 = time.perf_counter()
gj_out = generic_join_query(R1,R2,R3,R4,R5,R6,R7)
t1 = time.perf_counter()
generic_time = t1 - t0

# GHW Join
t0 = time.perf_counter()
ghw_out = ghw_join(R1,R2,R3,R4,R5,R6,R7)
t1 = time.perf_counter()
ghw_time = t1 - t0

# FHW Join
t0 = time.perf_counter()
fhw_out = fhw_join(R1,R2,R3,R4,R5,R6,R7)
t1 = time.perf_counter()
fhw_time = t1 - t0

generic_time, ghw_time, fhw_time, len(gj_out), len(ghw_out), len(fhw_out)


(10.63165729958564,
 0.3786563999019563,
 0.36425350001081824,
 1000000,
 1000000,
 1000000)